In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
###############################################
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [49]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_train.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [3]:
df_train.columns

Index(['id', 'Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'NObeyesdad'], dtype='object')

In [50]:
df_train = df_train.rename(columns={'FAVC': 'High_Calorie_Freq'
                                ,'FCVC': 'Vegetable_Freq',
                                  'NCP':'Main_Meals_No',
                                  'CAEC':'Eat_Between_Meals',
                                  'CH2O':'Water_Consumption',
                                  'FAF':'Physical_Activitiy_Freq',
                                  'TUE':'Time_Using_Devices',
                                  'MTRANS':'Transportation_Used',
                                  'CALC':'Alcohol_Consumption'})

In [5]:
df_train.columns

Index(['id', 'Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight', 'High_Calorie_Freq', 'Vegetable_Freq', 'Main_Meals_No', 'Eat_Between_Meals', 'SMOKE', 'Water_Consumption', 'SCC', 'Physical_Activitiy_Freq', 'Time_Using_Devices', 'Alcohol_Consumption', 'Transportation_Used', 'NObeyesdad'], dtype='object')

In [6]:
df_train['SCC'].value_counts()

SCC
no     20071
yes      687
Name: count, dtype: int64

In [51]:
df_train = df_train.drop('SCC', axis=1)
df_train.head()

In [51]:
df_train.columns

In [53]:
df_test['CALC'] = df_test['CALC'].replace('Always', 'Frequently')

In [54]:
df_train["Alcohol_Consumption"].value_counts()

Alcohol_Consumption
Sometimes     15066
no             5163
Frequently      529
Name: count, dtype: int64

In [55]:
alcohol_mapping = {'no': 0, 'Sometimes': 1, 'Frequently': 2}
df_train['Alcohol_Consumption'] = df_train['Alcohol_Consumption'].map(alcohol_mapping)
df_train.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,High_Calorie_Freq,Vegetable_Freq,Main_Meals_No,Eat_Between_Meals,SMOKE,Water_Consumption,Physical_Activitiy_Freq,Time_Using_Devices,Alcohol_Consumption,Transportation_Used,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,0.000000,0.976473,1,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,1.000000,1.000000,0,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,0.866045,1.673584,0,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,1.467863,0.780199,1,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,1.967973,0.931721,1,Public_Transportation,Overweight_Level_II


In [56]:
df_train["Alcohol_Consumption"].value_counts()

Series([], Name: count, dtype: int64)

In [19]:
#Feature Extraction

df_train['BMI'] = df_train['Weight'] / (df_train['Height']) ** 2
df_train['Hydration_Ratio'] = (df_train['Water_Consumption'] / df_train['Weight'])*100
df_train['Technology_Usage_Ratio'] = (df_train['Time_Using_Devices'] / df_train['Age'])*100
df_train['Alcohol_Consumption_Ratio'] = (df_train['Alcohol_Consumption'] / df_train['Age'])*100

In [20]:
df_train["High_Calorie_Freq"].value_counts()

calorie_mapping = {'no': 0, 'yes': 1}
df_train['High_Calorie_Freq'] = df_train['High_Calorie_Freq'].map(calorie_mapping)
df_train.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,High_Calorie_Freq,Vegetable_Freq,Main_Meals_No,Eat_Between_Meals,SMOKE,Water_Consumption,Physical_Activitiy_Freq,Time_Using_Devices,Alcohol_Consumption,Transportation_Used,NObeyesdad,BMI,Hydration_Ratio,Technology_Usage_Ratio,Alcohol_Consumption_Ratio
0,0,Male,24.443011,1.699998,81.669950,yes,1,2.000000,2.983297,Sometimes,no,2.763573,0.000000,0.976473,1,Public_Transportation,Overweight_Level_II,28.259565,3.383831,3.994897,4.091149
1,1,Female,18.000000,1.560000,57.000000,yes,1,2.000000,3.000000,Frequently,no,2.000000,1.000000,1.000000,0,Automobile,Normal_Weight,23.422091,3.508772,5.555556,0.000000
2,2,Female,18.000000,1.711460,50.165754,yes,1,1.880534,1.411685,Sometimes,no,1.910378,0.866045,1.673584,0,Public_Transportation,Insufficient_Weight,17.126706,3.808132,9.297689,0.000000
3,3,Female,20.952737,1.710730,131.274851,yes,1,3.000000,3.000000,Sometimes,no,1.674061,1.467863,0.780199,1,Public_Transportation,Obesity_Type_III,44.855798,1.275234,3.723614,4.772646
4,4,Male,31.641081,1.914186,93.798055,yes,1,2.679664,1.971472,Sometimes,no,1.979848,1.967973,0.931721,1,Public_Transportation,Overweight_Level_II,25.599151,2.110756,2.944656,3.160448


In [21]:
df_train["Eat_Between_Meals"].value_counts()

df_train['Eat_Between_Meals'] = df_train['Eat_Between_Meals'].map(alcohol_mapping)
df_train.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,High_Calorie_Freq,Vegetable_Freq,Main_Meals_No,Eat_Between_Meals,SMOKE,Water_Consumption,Physical_Activitiy_Freq,Time_Using_Devices,Alcohol_Consumption,Transportation_Used,NObeyesdad,BMI,Hydration_Ratio,Technology_Usage_Ratio,Alcohol_Consumption_Ratio
0,0,Male,24.443011,1.699998,81.669950,yes,1,2.000000,2.983297,1.0,no,2.763573,0.000000,0.976473,1,Public_Transportation,Overweight_Level_II,28.259565,3.383831,3.994897,4.091149
1,1,Female,18.000000,1.560000,57.000000,yes,1,2.000000,3.000000,2.0,no,2.000000,1.000000,1.000000,0,Automobile,Normal_Weight,23.422091,3.508772,5.555556,0.000000
2,2,Female,18.000000,1.711460,50.165754,yes,1,1.880534,1.411685,1.0,no,1.910378,0.866045,1.673584,0,Public_Transportation,Insufficient_Weight,17.126706,3.808132,9.297689,0.000000
3,3,Female,20.952737,1.710730,131.274851,yes,1,3.000000,3.000000,1.0,no,1.674061,1.467863,0.780199,1,Public_Transportation,Obesity_Type_III,44.855798,1.275234,3.723614,4.772646
4,4,Male,31.641081,1.914186,93.798055,yes,1,2.679664,1.971472,1.0,no,1.979848,1.967973,0.931721,1,Public_Transportation,Overweight_Level_II,25.599151,2.110756,2.944656,3.160448


In [22]:
df_train["SMOKE"].value_counts()

df_train['SMOKE'] = df_train['SMOKE'].map(calorie_mapping)
df_train.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,High_Calorie_Freq,Vegetable_Freq,Main_Meals_No,Eat_Between_Meals,SMOKE,Water_Consumption,Physical_Activitiy_Freq,Time_Using_Devices,Alcohol_Consumption,Transportation_Used,NObeyesdad,BMI,Hydration_Ratio,Technology_Usage_Ratio,Alcohol_Consumption_Ratio
0,0,Male,24.443011,1.699998,81.669950,yes,1,2.000000,2.983297,1.0,0,2.763573,0.000000,0.976473,1,Public_Transportation,Overweight_Level_II,28.259565,3.383831,3.994897,4.091149
1,1,Female,18.000000,1.560000,57.000000,yes,1,2.000000,3.000000,2.0,0,2.000000,1.000000,1.000000,0,Automobile,Normal_Weight,23.422091,3.508772,5.555556,0.000000
2,2,Female,18.000000,1.711460,50.165754,yes,1,1.880534,1.411685,1.0,0,1.910378,0.866045,1.673584,0,Public_Transportation,Insufficient_Weight,17.126706,3.808132,9.297689,0.000000
3,3,Female,20.952737,1.710730,131.274851,yes,1,3.000000,3.000000,1.0,0,1.674061,1.467863,0.780199,1,Public_Transportation,Obesity_Type_III,44.855798,1.275234,3.723614,4.772646
4,4,Male,31.641081,1.914186,93.798055,yes,1,2.679664,1.971472,1.0,0,1.979848,1.967973,0.931721,1,Public_Transportation,Overweight_Level_II,25.599151,2.110756,2.944656,3.160448


In [23]:
df_train["family_history_with_overweight"].value_counts()

df_train['family_history_with_overweight'] = df_train['family_history_with_overweight'].map(calorie_mapping)
df_train.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,High_Calorie_Freq,Vegetable_Freq,Main_Meals_No,Eat_Between_Meals,SMOKE,Water_Consumption,Physical_Activitiy_Freq,Time_Using_Devices,Alcohol_Consumption,Transportation_Used,NObeyesdad,BMI,Hydration_Ratio,Technology_Usage_Ratio,Alcohol_Consumption_Ratio
0,0,Male,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,1.0,0,2.763573,0.000000,0.976473,1,Public_Transportation,Overweight_Level_II,28.259565,3.383831,3.994897,4.091149
1,1,Female,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,2.0,0,2.000000,1.000000,1.000000,0,Automobile,Normal_Weight,23.422091,3.508772,5.555556,0.000000
2,2,Female,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,1.0,0,1.910378,0.866045,1.673584,0,Public_Transportation,Insufficient_Weight,17.126706,3.808132,9.297689,0.000000
3,3,Female,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,1.0,0,1.674061,1.467863,0.780199,1,Public_Transportation,Obesity_Type_III,44.855798,1.275234,3.723614,4.772646
4,4,Male,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,1.0,0,1.979848,1.967973,0.931721,1,Public_Transportation,Overweight_Level_II,25.599151,2.110756,2.944656,3.160448


In [24]:
transportation_order = {'Walking': 1, 'Bike': 2, 'Motorbike': 3, 'Automobile': 4, 'Public_Transportation': 5}
df_train['Transportation_Used_Ordinal_Encoded'] = df_train['Transportation_Used'].map(transportation_order)
df_train.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,High_Calorie_Freq,Vegetable_Freq,Main_Meals_No,Eat_Between_Meals,SMOKE,Water_Consumption,Physical_Activitiy_Freq,Time_Using_Devices,Alcohol_Consumption,Transportation_Used,NObeyesdad,BMI,Hydration_Ratio,Technology_Usage_Ratio,Alcohol_Consumption_Ratio,Transportation_Used_Ordinal_Encoded
0,0,Male,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,1.0,0,2.763573,0.000000,0.976473,1,Public_Transportation,Overweight_Level_II,28.259565,3.383831,3.994897,4.091149,5
1,1,Female,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,2.0,0,2.000000,1.000000,1.000000,0,Automobile,Normal_Weight,23.422091,3.508772,5.555556,0.000000,4
2,2,Female,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,1.0,0,1.910378,0.866045,1.673584,0,Public_Transportation,Insufficient_Weight,17.126706,3.808132,9.297689,0.000000,5
3,3,Female,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,1.0,0,1.674061,1.467863,0.780199,1,Public_Transportation,Obesity_Type_III,44.855798,1.275234,3.723614,4.772646,5
4,4,Male,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,1.0,0,1.979848,1.967973,0.931721,1,Public_Transportation,Overweight_Level_II,25.599151,2.110756,2.944656,3.160448,5


In [25]:
df_train = df_train.drop('Transportation_Used', axis=1)
df_train.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,High_Calorie_Freq,Vegetable_Freq,Main_Meals_No,Eat_Between_Meals,SMOKE,Water_Consumption,Physical_Activitiy_Freq,Time_Using_Devices,Alcohol_Consumption,NObeyesdad,BMI,Hydration_Ratio,Technology_Usage_Ratio,Alcohol_Consumption_Ratio,Transportation_Used_Ordinal_Encoded
0,0,Male,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,1.0,0,2.763573,0.000000,0.976473,1,Overweight_Level_II,28.259565,3.383831,3.994897,4.091149,5
1,1,Female,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,2.0,0,2.000000,1.000000,1.000000,0,Normal_Weight,23.422091,3.508772,5.555556,0.000000,4
2,2,Female,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,1.0,0,1.910378,0.866045,1.673584,0,Insufficient_Weight,17.126706,3.808132,9.297689,0.000000,5
3,3,Female,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,1.0,0,1.674061,1.467863,0.780199,1,Obesity_Type_III,44.855798,1.275234,3.723614,4.772646,5
4,4,Male,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,1.0,0,1.979848,1.967973,0.931721,1,Overweight_Level_II,25.599151,2.110756,2.944656,3.160448,5


In [26]:
df_train['Gender_Binary'] = df_train['Gender'].map({'Male': 0, 'Female': 1})
df_train = df_train.drop('Gender', axis=1)
df_train.head()

,id,Age,Height,Weight,family_history_with_overweight,High_Calorie_Freq,Vegetable_Freq,Main_Meals_No,Eat_Between_Meals,SMOKE,Water_Consumption,Physical_Activitiy_Freq,Time_Using_Devices,Alcohol_Consumption,NObeyesdad,BMI,Hydration_Ratio,Technology_Usage_Ratio,Alcohol_Consumption_Ratio,Transportation_Used_Ordinal_Encoded,Gender_Binary
0,0,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,1.0,0,2.763573,0.000000,0.976473,1,Overweight_Level_II,28.259565,3.383831,3.994897,4.091149,5,0
1,1,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,2.0,0,2.000000,1.000000,1.000000,0,Normal_Weight,23.422091,3.508772,5.555556,0.000000,4,1
2,2,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,1.0,0,1.910378,0.866045,1.673584,0,Insufficient_Weight,17.126706,3.808132,9.297689,0.000000,5,1
3,3,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,1.0,0,1.674061,1.467863,0.780199,1,Obesity_Type_III,44.855798,1.275234,3.723614,4.772646,5,1
4,4,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,1.0,0,1.979848,1.967973,0.931721,1,Overweight_Level_II,25.599151,2.110756,2.944656,3.160448,5,0


In [27]:
x = df_train.drop('NObeyesdad', axis=1)  # Features (all columns except 'NObeyesdad')
y = df_train['NObeyesdad']  # Target variable

In [28]:
from sklearn.preprocessing import LabelEncoder , StandardScaler
from sklearn.model_selection import train_test_split
le=LabelEncoder()
sc=StandardScaler()

In [29]:
y=le.fit_transform(y)

In [30]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42)

In [42]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
# Set random hyperparameters
params = {
    'max_depth': 10,
    'learning_rate': 0.01,
    'n_estimators': 400,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'gamma': 0,
    'random_state': 42
}

# Initialize XGBClassifier
model = xgb.XGBClassifier(**params, objective='multi:softmax', num_class=7) 
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_val)

# Evaluate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9004495825305073


In [44]:
df_test.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation


In [46]:
ids=df_test['id']

In [47]:
df_test=df_test.drop('id',axis=1)